# Libraries

In [10]:
!pip install transformers datasets accelerate peft sentencepiece

In [ ]:
from google.colab import drive
import json
import os
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import Dataset
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)

# Loading Data

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Testing The Model Before Training

In [ ]:
# غیرفعال‌سازی W&B
os.environ["WANDB_DISABLED"] = "true"

# لیبل‌ها (فارسی)
labels_list = ['اقتصادی', 'تفریح و نشاط', 'فرهنگ', 'ورزشی']
label2id = {label: i for i, label in enumerate(labels_list)}
id2label = {i: label for label, i in label2id.items()}

# بارگذاری مدل خام و توکنایزر
MODEL_NAME = "HooshvareLab/bert-fa-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(labels_list),
    id2label=id2label,
    label2id=label2id
)

# دریافت ورودی از کاربر
print("لطفاً یک یا چند متن فارسی وارد کنید (برای پایان Enter خالی بزنید):")
texts = []
while True:
    line = input("متن: ").strip()
    if line == "":
        break
    texts.append(line)

if not texts:
    print("متنی وارد نشده.")
    exit()

# پیش‌پردازش
inputs = tokenizer(
    texts,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=256
)

# پیش‌بینی
with torch.no_grad():
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    preds = torch.argmax(probs, dim=1)

# چاپ نتایج
print("\nنتایج پیش‌بینی مدل خام:")
for i, text in enumerate(texts):
    label = id2label[preds[i].item()]
    confidence = probs[i][preds[i]].item()
    print(f"متن: {text}")
    print(f"دسته‌بندی پیش‌بینی‌شده: {label} (اطمینان: {confidence:.2%})\n")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--HooshvareLab--bert-fa-base-uncased/snapshots/a04aa40c97bcdde570ae11986a534542c2995a62/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.53.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--HooshvareLab--bert-fa-base-uncased/snapshots/a04aa40c97bcd

لطفاً یک یا چند متن فارسی وارد کنید (برای پایان Enter خالی بزنید):


Safetensors PR exists


متن:  در پی نوسانات شدید بازار ارز و افزایش نرخ دلار در هفته‌های اخیر، بانک مرکزی با اتخاذ سیاست‌های جدیدی تلاش کرده است تا ثبات نسبی را به بازار ارز بازگرداند. در این راستا، عرضه ارز در سامانه نیما افزایش یافته و سقف خرید ارز برای واردکنندگان کاهش یافته است. همچنین، برنامه‌هایی برای مدیریت تقاضای ارز مسافرتی و تجاری در دست اجراست. کارشناسان معتقدند که تنها راهکار بلندمدت برای مهار تورم و کنترل نوسانات ارزی، تقویت تولید داخلی، جذب سرمایه‌گذاری خارجی و افزایش صادرات غیرنفتی است. در همین حال، بخش خصوصی نیز خواستار کاهش موانع تجاری و اصلاح ساختار بانکی برای تسهیل فعالیت‌های اقتصادی شده است.
متن:  در مراسم افتتاحیه سی‌ونهمین جشنواره بین‌المللی فیلم فجر، که با حضور گسترده اهالی سینما، منتقدان و خبرنگاران برگزار شد، آثار شاخصی از سینماگران جوان و باتجربه به نمایش درآمد. امسال با توجه به تغییرات در آیین‌نامه داوری و افزایش تعداد فیلم‌های بخش مسابقه، تنوع بیشتری در ژانرها و سبک‌ها مشاهده می‌شود. همچنین، فیلم‌هایی با رویکرد اجتماعی و پرداختن به مسائل روز جامعه ایرانی بیشتر مورد توجه قرار گرفته‌

# LORA

In [ ]:

# ==== تنظیمات ====
MODEL_NAME = "HooshvareLab/bert-fa-base-uncased"
TRAIN_FILE = "/content/drive/MyDrive/Final/lora_train.json"
VAL_FILE = "/content/drive/MyDrive/Final/lora_val.json"
OUTPUT_DIR = "/content/drive/MyDrive/Final/final-bert-fa-lora"

NUM_EPOCHS = 3
BATCH_SIZE = 8
SEED = 42

torch.manual_seed(SEED)
np.random.seed(SEED)

# ==== بارگذاری داده‌ها ====
with open(TRAIN_FILE, "r", encoding="utf-8") as f:
    train_data = json.load(f)
with open(VAL_FILE, "r", encoding="utf-8") as f:
    val_data = json.load(f)

labels_list = sorted(list(set(item['label'] for item in train_data + val_data)))
label2id = {label: i for i, label in enumerate(labels_list)}
id2label = {i: label for label, i in label2id.items()}
NUM_LABELS = len(labels_list)

# ==== لیبل‌های انگلیسی فقط برای نمایش در نمودار ====
# ترتیب باید با labels_list یکسان باشد
label_translation = {
    'اقتصادی': 'Economic',
    'تفریح و نشاط': 'Entertainment',
    'فرهنگ': 'Culture',
    'ورزشی': 'Sports'
}
labels_list_en = [label_translation[label] for label in labels_list]

# ==== توکنایزر ====
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    text = examples["text"]
    labels = [label2id[label] for label in examples["label"]]
    encodings = tokenizer(
        text,
        max_length=256,
        padding="max_length",
        truncation=True
    )
    encodings["labels"] = labels
    return encodings

train_dataset = Dataset.from_list(train_data).map(preprocess_function, batched=True)
val_dataset = Dataset.from_list(val_data).map(preprocess_function, batched=True)

train_dataset = train_dataset.remove_columns(["text", "label"])
val_dataset = val_dataset.remove_columns(["text", "label"])

# ==== مدل ====
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    id2label=id2label,
    label2id=label2id
)

# ==== تابع متریک ====
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

# ==== تنظیمات آموزش ====
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=NUM_EPOCHS,
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_strategy="epoch",
    report_to="none",
    log_level="info",
    save_strategy="epoch",
    save_total_limit=2,
    seed=SEED,
    do_eval=True,
    load_best_model_at_end=False,
    disable_tqdm=False,
    metric_for_best_model="accuracy",
    greater_is_better=True
)

# ==== Trainer ====
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

# ==== آموزش ====
print("🚀 شروع آموزش مدل...")
trainer.train()

# ==== ذخیره مدل ====
os.makedirs(OUTPUT_DIR, exist_ok=True)
trainer.save_model(os.path.join(OUTPUT_DIR, "best_model"))
print("✅ مدل با موفقیت ذخیره شد!")

# ==== پیش‌بینی و گزارش ====
predictions = trainer.predict(val_dataset)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

print("\n📊 Classification Report:")
print(classification_report(labels, preds, target_names=labels_list))

# ==== Confusion Matrix ====
cm = confusion_matrix(labels, preds)
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    xticklabels=labels_list_en,
    yticklabels=labels_list_en,
    cmap="Blues",
    linewidths=0.5,
    linecolor='gray'
)
plt.xlabel("Predicted", fontsize=12)
plt.ylabel("True", fontsize=12)
plt.title("Confusion Matrix", fontsize=14)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "confusion_matrix.png"))
plt.show()

# ==== ذخیره نتایج در فایل JSON ====
results_data = {
    "train_loss": float(trainer.state.log_history[-1]["train_loss"]) if "train_loss" in trainer.state.log_history[-1] else 0,
    "val_accuracy": float(accuracy_score(labels, preds)),
    "classification_report": classification_report(labels, preds, target_names=labels_list, output_dict=True),
    "confusion_matrix": cm.tolist(),
    "labels": labels_list
}

results_path = os.path.join(OUTPUT_DIR, "results.json")
with open(results_path, "w", encoding="utf-8") as f:
    json.dump(results_data, f, ensure_ascii=False, indent=4)

print(f"✅ نتایج در مسیر زیر ذخیره شد: {results_path}")


In [ ]:
import random
# ==== تنظیمات ====
MODEL_NAME = "HooshvareLab/bert-fa-base-uncased"
TRAIN_FILE = "/content/drive/MyDrive/Final/lora_train.json"
VAL_FILE = "/content/drive/MyDrive/Final/lora_val.json"
OUTPUT_DIR = "/content/drive/MyDrive/Final1/final-bert-fa-lora"

NUM_EPOCHS = 10
BATCH_SIZE = 8
SEED = 42

random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

# ==== بارگذاری داده‌ها ====
with open(TRAIN_FILE, "r", encoding="utf-8") as f:
    train_data = json.load(f)
with open(VAL_FILE, "r", encoding="utf-8") as f:
    val_data = json.load(f)

labels_list = sorted(list(set(item['label'] for item in train_data + val_data)))
label2id = {label: i for i, label in enumerate(labels_list)}
id2label = {i: label for label, i in label2id.items()}
NUM_LABELS = len(labels_list)

label_translation = {
    'اقتصادی': 'Economic',
    'تفریح و نشاط': 'Entertainment',
    'فرهنگ': 'Culture',
    'ورزشی': 'Sports'
}
labels_list_en = [label_translation[label] for label in labels_list]

# ==== توکنایزر ====
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    text = examples["text"]
    labels = [label2id[label] for label in examples["label"]]
    encodings = tokenizer(
        text,
        max_length=256,
        padding="max_length",
        truncation=True
    )
    encodings["labels"] = labels
    return encodings

train_dataset = Dataset.from_list(train_data).map(preprocess_function, batched=True)
val_dataset = Dataset.from_list(val_data).map(preprocess_function, batched=True)

train_dataset = train_dataset.remove_columns(["text", "label"])
val_dataset = val_dataset.remove_columns(["text", "label"])

# ==== مدل با Dropout بیشتر ====
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    id2label=id2label,
    label2id=label2id,
    hidden_dropout_prob=0.3,     # ← افزایش Dropout
    attention_probs_dropout_prob=0.3  # ← Dropout در توجه
)

# ==== تابع متریک ====
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

# ==== تنظیمات آموزش ====
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=2e-5,
    weight_decay=0.01,  # ← کمک به جلوگیری از overfitting
    num_train_epochs=NUM_EPOCHS,
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_strategy="epoch",
    eval_strategy="epoch",  # ← برای EarlyStopping لازم است
    save_strategy="epoch",
    save_total_limit=2,
    seed=SEED,
    do_eval=True,
    load_best_model_at_end=True,  # ← بهترین مدل ذخیره می‌شود
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to="none",
    log_level="info"
)

# ==== Trainer با EarlyStopping ====
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # ← توقف زودهنگام
)

# ==== آموزش ====
print("🚀 شروع آموزش مدل...")
trainer.train()

# ==== ذخیره بهترین مدل ====
os.makedirs(OUTPUT_DIR, exist_ok=True)
trainer.save_model(os.path.join(OUTPUT_DIR, "best_model"))
print("✅ مدل با موفقیت ذخیره شد!")

# ==== ارزیابی ====
predictions = trainer.predict(val_dataset)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids

print("\n📊 Classification Report:")
print(classification_report(labels, preds, target_names=labels_list))

# ==== Confusion Matrix ====
cm = confusion_matrix(labels, preds)
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    xticklabels=labels_list_en,
    yticklabels=labels_list_en,
    cmap="Blues",
    linewidths=0.5,
    linecolor='gray'
)
plt.xlabel("Predicted", fontsize=12)
plt.ylabel("True", fontsize=12)
plt.title("Confusion Matrix", fontsize=14)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "confusion_matrix.png"))
plt.show()

# ==== ذخیره نتایج در JSON ====
results_data = {
    "train_loss": float(trainer.state.log_history[-1].get("train_loss", 0)),
    "val_accuracy": float(accuracy_score(labels, preds)),
    "classification_report": classification_report(labels, preds, target_names=labels_list, output_dict=True),
    "confusion_matrix": cm.tolist(),
    "labels": labels_list
}

results_path = os.path.join(OUTPUT_DIR, "results.json")
with open(results_path, "w", encoding="utf-8") as f:
    json.dump(results_data, f, ensure_ascii=False, indent=4)

print(f"✅ نتایج در مسیر زیر ذخیره شد: {results_path}")


# Test

In [ ]:


# ==== غیرفعال‌سازی W&B ====
os.environ["WANDB_DISABLED"] = "true"

# ==== مسیرها ====
MODEL_PATH = "/content/drive/MyDrive/Final/final-bert-fa-lora/best_model"
TEST_FILE = "/content/drive/MyDrive/Final/lora_test.json"

# ==== لیبل‌ها (فارسی برای دسته‌بندی) ====
labels_list = ['اقتصادی', 'تفریح و نشاط', 'فرهنگ', 'ورزشی']
label2id = {label: i for i, label in enumerate(labels_list)}
id2label = {i: label for label, i in label2id.items()}

# ==== بارگذاری مدل ====
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    local_files_only=True,
    num_labels=len(labels_list),
    id2label=id2label,
    label2id=label2id
)

# ==== بارگذاری داده تست ====
with open(TEST_FILE, "r", encoding="utf-8") as f:
    test_data = json.load(f)

test_data_cleaned = [item for item in test_data if item.get("label") in label2id]
removed_count = len(test_data) - len(test_data_cleaned)
print(f"⚠️ {removed_count} نمونه حذف شدند به‌دلیل لیبل نادرست یا خالی.")

# ==== پیش‌پردازش ====
def preprocess_function(examples):
    texts = examples["text"]
    labels = [label2id[label] for label in examples["label"]]
    encodings = tokenizer(
        texts,
        max_length=256,
        padding="max_length",
        truncation=True
    )
    encodings["labels"] = labels
    return encodings

test_dataset = Dataset.from_list(test_data_cleaned).map(preprocess_function, batched=True)
test_dataset = test_dataset.remove_columns(["text", "label"])

# ==== متریک ====
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

# ==== Trainer ====
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics
)

# ==== ارزیابی ====
print("🔍 در حال ارزیابی مدل روی داده‌های تست...")
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)
labels = predictions.label_ids
metrics = predictions.metrics

# ==== نتایج ====
print("\n📉 Test Loss:", round(metrics["test_loss"], 4))
print("✅ Test Accuracy:", round(metrics["test_accuracy"], 4))

print("\n📊 Classification Report:")
print(classification_report(labels, preds, target_names=labels_list))

# ==== Confusion Matrix ====
labels_list_en = ['Economic', 'Entertainment', 'Culture', 'Sports']

cm = confusion_matrix(labels, preds)
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    xticklabels=labels_list_en,
    yticklabels=labels_list_en,
    cmap="YlGnBu",
    linewidths=0.5,
    linecolor='gray'
)
plt.xlabel("Predicted Labels", fontsize=12)
plt.ylabel("True Labels", fontsize=12)
plt.title("Confusion Matrix - Test Set", fontsize=14)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


# Unseen Data(test)

In [ ]:
model_path = "/content/drive/MyDrive/Final1/final-bert-fa-lora/best_model"

# ===== بارگذاری مدل و توکنایزر =====
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)


label_names = ["اقتصادی", "تفریح و نشاط", "فرهنگ", "ورزشی"]

# ===== تابع پیش‌بینی =====
def classify_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1)
        pred_class = torch.argmax(probs, dim=-1).item()
        confidence = probs[0][pred_class].item()

    print(f"\n📝 متن: {text}")
    print(f"📚 برچسب پیش‌بینی‌شده: {label_names[pred_class]}")
    print(f"📊 اعتماد مدل: {confidence:.2%}")

# ===== تست با یک ورودی دلخواه =====
while True:
    user_input = input("\n🔹 یک متن وارد کن (یا 'exit' برای خروج):\n> ")
    if user_input.strip().lower() == "exit":
        break
    classify_text(user_input)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja
loading configuration file /content/drive/MyDrive/Final1/final-bert-fa-lora/best_model/config.json
Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.3,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.3,
  "hidden_size": 768,
  "id2label": {
    "0": "\u0627\u0642\u062a\u0635\u0627\u062f\u06cc",
    "1": "\u062a\u0641\u0631\u06cc\u062d \u0648 \u0646\u0634\u0627\u0637",
    "2": "\u0641\u0631\u0647\u0646\u06af",
    "3": "\u0648\u0631\u0632\u0634\u06cc"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "\u0627\u0642\u062a\u0635\u0627\u062f\u06cc": 0,
    "\u062a\u0641\u0631\u06cc\u062d \u0648 \u0646\u0634\u0627\u0637": 1,
    "\u0641\u0631\u0647\u0646\u06af": 2,
    


🔹 یک متن وارد کن (یا 'exit' برای خروج):
>  در پی نوسانات شدید بازار ارز و افزایش نرخ دلار در هفته‌های اخیر، بانک مرکزی با اتخاذ سیاست‌های جدیدی تلاش کرده است تا ثبات نسبی را به بازار ارز بازگرداند. در این راستا، عرضه ارز در سامانه نیما افزایش یافته و سقف خرید ارز برای واردکنندگان کاهش یافته است. همچنین، برنامه‌هایی برای مدیریت تقاضای ارز مسافرتی و تجاری در دست اجراست. کارشناسان معتقدند که تنها راهکار بلندمدت برای مهار تورم و کنترل نوسانات ارزی، تقویت تولید داخلی، جذب سرمایه‌گذاری خارجی و افزایش صادرات غیرنفتی است. در همین حال، بخش خصوصی نیز خواستار کاهش موانع تجاری و اصلاح ساختار بانکی برای تسهیل فعالیت‌های اقتصادی شده است.


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



📝 متن:  در پی نوسانات شدید بازار ارز و افزایش نرخ دلار در هفته‌های اخیر، بانک مرکزی با اتخاذ سیاست‌های جدیدی تلاش کرده است تا ثبات نسبی را به بازار ارز بازگرداند. در این راستا، عرضه ارز در سامانه نیما افزایش یافته و سقف خرید ارز برای واردکنندگان کاهش یافته است. همچنین، برنامه‌هایی برای مدیریت تقاضای ارز مسافرتی و تجاری در دست اجراست. کارشناسان معتقدند که تنها راهکار بلندمدت برای مهار تورم و کنترل نوسانات ارزی، تقویت تولید داخلی، جذب سرمایه‌گذاری خارجی و افزایش صادرات غیرنفتی است. در همین حال، بخش خصوصی نیز خواستار کاهش موانع تجاری و اصلاح ساختار بانکی برای تسهیل فعالیت‌های اقتصادی شده است.
📚 برچسب پیش‌بینی‌شده: اقتصادی
📊 اعتماد مدل: 99.39%

🔹 یک متن وارد کن (یا 'exit' برای خروج):
>  در مراسم افتتاحیه سی‌ونهمین جشنواره بین‌المللی فیلم فجر، که با حضور گسترده اهالی سینما، منتقدان و خبرنگاران برگزار شد، آثار شاخصی از سینماگران جوان و باتجربه به نمایش درآمد. امسال با توجه به تغییرات در آیین‌نامه داوری و افزایش تعداد فیلم‌های بخش مسابقه، تنوع بیشتری در ژانرها و سبک‌ها مشاهده می‌شود. همچنین،

# New Section

In [11]:
model_path = "/content/drive/MyDrive/Final1/final-bert-fa-lora/best_model"

# ===== بارگذاری مدل و توکنایزر =====
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)


label_names = ["اقتصادی", "تفریح و نشاط", "فرهنگ", "ورزشی"]

# ===== تابع پیش‌بینی =====
def classify_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1)
        pred_class = torch.argmax(probs, dim=-1).item()
        confidence = probs[0][pred_class].item()

    print(f"\n📝 متن: {text}")
    print(f"📚 برچسب پیش‌بینی‌شده: {label_names[pred_class]}")
    print(f"📊 اعتماد مدل: {confidence:.2%}")

# ===== تست با یک ورودی دلخواه =====
while True:
    user_input = input("\n🔹 یک متن وارد کن (یا 'exit' برای خروج):\n> ")
    if user_input.strip().lower() == "exit":
        break
    classify_text(user_input)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file chat_template.jinja
loading configuration file /content/drive/MyDrive/Final1/final-bert-fa-lora/best_model/config.json
Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.3,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.3,
  "hidden_size": 768,
  "id2label": {
    "0": "\u0627\u0642\u062a\u0635\u0627\u062f\u06cc",
    "1": "\u062a\u0641\u0631\u06cc\u062d \u0648 \u0646\u0634\u0627\u0637",
    "2": "\u0641\u0631\u0647\u0646\u06af",
    "3": "\u0648\u0631\u0632\u0634\u06cc"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "\u0627\u0642\u062a\u0635\u0627\u062f\u06cc": 0,
    "\u062a\u0641\u0631\u06cc\u062d \u0648 \u0646\u0634\u0627\u0637": 1,
    "\u0641\u0631\u0647\u0646\u06af": 2,
    


🔹 یک متن وارد کن (یا 'exit' برای خروج):
> نساجی مازندران در فصل جدید لیگ دسته اول تیمی پرمهره خواهد بود که برای بازگشت سریع به لیگ‌برتر تلاش می‌کند. اگرچه نساجی هنوز به‌صورت رسمی فراز کمالوند را به‌عنوان سرمربی اعلام نکرده، اما مدیران باشگاه با او به توافق نزدیک هستند و مذاکره برای جذب بازیکنان سرشناس را آغاز کرده‌اند. در روزهای اخیر، باشگاه تمرینات را در مازندران برگزار کرده و فعالیت‌های خود را برای تقویت تیم با جذب ستاره‌های لیگ‌برتر آغاز کرده است. مدیران باشگاه قصد دارند تیمی قدرتمند و پرمهره راه‌اندازی کنند تا نه‌تنها رضایت هواداران را جلب کنند،‌ بلکه با قرار گرفتن بین دو تیم برتر لیگ‌یک، بلافاصله به لیگ‌برتر بازگردند. طبق پیش‌بینی‌ها، در آینده‌ای نزدیک شاهد رونمایی رسمی باشگاه از چند بازیکن جدید و باتجربه خواهیم بود.


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



📝 متن: نساجی مازندران در فصل جدید لیگ دسته اول تیمی پرمهره خواهد بود که برای بازگشت سریع به لیگ‌برتر تلاش می‌کند. اگرچه نساجی هنوز به‌صورت رسمی فراز کمالوند را به‌عنوان سرمربی اعلام نکرده، اما مدیران باشگاه با او به توافق نزدیک هستند و مذاکره برای جذب بازیکنان سرشناس را آغاز کرده‌اند. در روزهای اخیر، باشگاه تمرینات را در مازندران برگزار کرده و فعالیت‌های خود را برای تقویت تیم با جذب ستاره‌های لیگ‌برتر آغاز کرده است. مدیران باشگاه قصد دارند تیمی قدرتمند و پرمهره راه‌اندازی کنند تا نه‌تنها رضایت هواداران را جلب کنند،‌ بلکه با قرار گرفتن بین دو تیم برتر لیگ‌یک، بلافاصله به لیگ‌برتر بازگردند. طبق پیش‌بینی‌ها، در آینده‌ای نزدیک شاهد رونمایی رسمی باشگاه از چند بازیکن جدید و باتجربه خواهیم بود.
📚 برچسب پیش‌بینی‌شده: ورزشی
📊 اعتماد مدل: 99.94%

🔹 یک متن وارد کن (یا 'exit' برای خروج):
> بر اساس گزارش رسمی خبرگزاری تسنیم، در خرداد ماه ۱۴۰۴ میزان عرضه گوشت قرمز در کشتارگاه‌های کشور حدود ۱۵ درصد نسبت به اردیبهشت ماه افزایش یافته است. این افزایش عرضه به‌عنوان یک شاخص مثبت در حوزه اقتصاد مواد غذایی